In [1]:
!pip install evaluation

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for evaluation: filename=evaluation-0.0.2-py3-none-any.whl size=2445 sha256=8e3ae3185c754d25dcceb527c42a302636b9656351d12d84cc003e599f119f1e
  Stored in directory: /root/.cache/pip/wheels/c2/1b/52/3a6b0472227ed68ab68fb46af7a1c87b11b6685b148b913ddb
  Created wheel for python-gflags: filename=python_gflags-3.1.2-py3-none-any.whl size=57371 sha256=1498d7b2538e694cb7c67c3b47d62185d02fb8a32dfdcdab60d9e2c6617eddde
  Stored in directory: /root/.cache/pip/wheels/05/9a/41/a8f105872405a3453769fb518071f61d27e355b10561461073
Successfully built evaluation python-gflags


In [3]:
# ===============================
# 1️⃣ Imports
# ===============================
import torch
import numpy as np
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


# ===============================
# 2️⃣ Load & Explore Dataset
# ===============================
dataset = load_dataset("sms_spam")

print("Column Names:", dataset["train"].column_names)
print("Label Names:", dataset["train"].features["label"].names)

# Expected:
# text column -> "text"
# label column -> "label"
# labels -> ['ham', 'spam']


# ===============================
# 3️⃣ Label Dictionaries
# ===============================
label_map = {0: "ham", 1: "spam"}
id_map = {"ham": 0, "spam": 1}


# ===============================
# 4️⃣ Tokenizer
# ===============================
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_function(example):
    return tokenizer(
        example["sms"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

tokenized_dataset = dataset.map(tokenize_function, batched=True)


# ===============================
# 5️⃣ Shuffle & Split Dataset
# ===============================
tokenized_dataset = tokenized_dataset.shuffle(seed=42)

train_dataset = tokenized_dataset["train"].select(range(4000))
eval_dataset = tokenized_dataset["train"].select(range(1000))

train_dataset.set_format(
    "torch",
    columns=["input_ids", "attention_mask", "label"]
)
eval_dataset.set_format(
    "torch",
    columns=["input_ids", "attention_mask", "label"]
)


# ===============================
# 6️⃣ Load Model
# ===============================
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2,
    id2label=label_map,
    label2id=id_map
)


# ===============================
# 7️⃣ Metrics
# ===============================
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)

    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average="binary"
    )
    accuracy = accuracy_score(labels, predictions)

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }


# ===============================
# 8️⃣ Training Arguments
# ===============================
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="f1"
)


# ===============================
# 9️⃣ Trainer
# ===============================
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


# ===============================
# 🔟 Train Model
# ===============================
trainer.train()


# ===============================
# 1️⃣1️⃣ Save Model
# ===============================
trainer.save_model("./spam_model")
tokenizer.save_pretrained("./spam_model")


# ===============================
# 1️⃣2️⃣ Load Model for Inference
# ===============================
tokenizer = AutoTokenizer.from_pretrained("./spam_model")
model = AutoModelForSequenceClassification.from_pretrained("./spam_model")
model.eval()


# ===============================
# 1️⃣3️⃣ Prediction Function
# ===============================
def predict_with_label(text):
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=128
    )

    with torch.no_grad():
        outputs = model(**inputs)

    pred_id = torch.argmax(outputs.logits, dim=1).item()
    return label_map[pred_id]


# ===============================
# 1️⃣4️⃣ Test Examples
# ===============================
texts = [
    "Congratulations! You've won a free ticket.",
    "Hey, are we meeting tomorrow?",
]

for t in texts:
    print("Text:", t)
    print("Prediction:", predict_with_label(t))
    print("-" * 50)


Column Names: ['sms', 'label']
Label Names: ['ham', 'spam']


Map:   0%|          | 0/5574 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-4232674931.py:124: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.045900,0.022379,0.995000,0.976000,0.983871,0.979920
2,0.023200,0.013517,0.998000,1.000000,0.983871,0.991870


Text: Congratulations! You've won a free ticket.
Prediction: ham
--------------------------------------------------
Text: Hey, are we meeting tomorrow?
Prediction: ham
--------------------------------------------------
